In [2]:
import couchdb
import elasticsearch

### Passo a passo:
1. Recuperar do ES todos os documentos que contêm Raquel
+ Para cada documento, verificar se é Raquel ou Raquel Negrão
+ Se contiver somente Raquel Negrão, não fazer nada
+ Se contiver Raquel e contiver Raquel Negrão, deletar Raquel
+ Se somente contiver Raquel, substituir Raquel por Raquel Negrão

In [3]:
# Set ES connection
es = elasticsearch.Elasticsearch('http://10.10.100.48/')
es.ping()

True

In [22]:
# Set CouchDB connection
couch = couchdb.Server('http://10.10.100.48/couchdb')

In [23]:
body = """
{
    "query": {"match": {
       "metadata.contributor": "raquel"
       }
    }
}
"""

In [24]:
docs = es.search(index='_all', body=body, size=5000)
print "Got %d documents" % len(docs['hits']['hits'])

Got 4741 documents


In [28]:
only_negrao = []
only_raquel = []
both = []
no_raquel = []
not_found = []
only_one_email = []
not_all_email = []
antes = []
depois = []
i = 0

for doc in docs['hits']['hits']:
    update=False
    # Get alias or index name
    aliases = es.indices.get_aliases(index=doc["_index"])
    if len(aliases[doc["_index"]]["aliases"]) == 0:
        doc_index = doc["_index"]
    else:
        doc_index = aliases[doc["_index"]]["aliases"].keys()[0]
    doc_id = doc['_id']
    
    # Get document
    try:
        db = couch[doc_index]
    except couchdb.ServerError:
        print doc_index
        break
    doc_couch = db.get(doc_id)
    if doc_couch == None:
        not_found.append(doc_id)
        continue
        
    # Get fields
    contributor = doc_couch['metadata']['contributor']
    contact = doc_couch['metadata']['contact']
    contributor_array = contributor.split(';')
    contact_array = contact.split(';')
    contributor_array = [unicode(person.strip()) for person in contributor_array]
    contact_array = [unicode(person.strip()) for person in contact_array]
    
    # Check if contact field has only emails
    all_email = True
    for email in contact_array:
        if '@' not in email and email != '':
            all_email = False
            break
    if not all_email:
        not_all_email.append(doc_id)
        continue
    
    if u'Raquel' not in contributor_array and u'Raquel Negrão' in contributor_array:
        only_negrao.append(doc_id)
    elif u'Raquel' in contributor_array and u'Raquel Negrão' in contributor_array:
        both.append(doc_id)
        raquel_index = contributor_array.index('Raquel')
        del contributor_array[raquel_index]
        try:
            del contact_array[raquel_index]
        except IndexError:
            only_one_email.append(doc_id)
        doc_couch['metadata']['contributor'] = " ; ".join(contributor_array)
        doc_couch['metadata']['contact'] = " ; ".join(contact_array)
        update = True
    elif u'Raquel' in contributor_array and u'Raquel Negrão' not in contributor_array:
        only_raquel.append(doc_id)
        raquel_index = contributor_array.index('Raquel')
        contributor_array[raquel_index] = u'Raquel Negrão'
        doc_couch['metadata']['contributor'] = " ; ".join(contributor_array)
        update = True
    else:
        # Shouldn't happen
        no_raquel.append(doc_id)
    #if update:
        #db.update([doc_couch])   
    

In [29]:
print "Only Negrao: %d" % len(only_negrao)
print "Only Raquel: %d" % len(only_raquel)
print "Both: %d" % len(both)
print "Only one email: %d" % len(only_one_email)
print "None: %d" % len(no_raquel)
print "Ids not found: %d" % len(not_found)
print "Not all email: %d" % len(not_all_email)

Only Negrao: 4731
Only Raquel: 0
Both: 0
Only one email: 0
None: 0
Ids not found: 10
Not all email: 0


### Consertando docs que não são recuperados pelo id

In [22]:
for doc in docs['hits']['hits']:
    # Get alias or index name
    aliases = es.indices.get_aliases(index=doc["_index"])
    if len(aliases[doc["_index"]]["aliases"]) == 0:
        doc_index = doc["_index"]
    else:
        doc_index = aliases[doc["_index"]]["aliases"].keys()[0]
    doc_id = doc['_id']
    
    # Get document
    try:
        db = couch[doc_index]
    except couchdb.ServerError:
        print doc_index
        break
    doc_couch = db.get(doc_id)
    if doc_couch == None:
        print doc
        break

{u'_score': 2.8374934, u'_type': u'occurrence', u'_id': u'Arbusto c/ 0,5 m de alt. ramos verdes, folhas cori\xe1ceas discolor verde escuras, ped\xfanculos lil\xe1s, flores roxas', u'_source': {u'family': u'Faramea calyciflora A . Rich.', u'institutionCode': u'2014-07-22', u'municipality': u'Rio de Janeiro', u'month': u'1992', u'identifiedBy': u'A . Rich.', u'acceptedNameUsage': u'occurrence:rj:4648900', u'year': u'Gomes, M.', u'id': u'Arbusto c/ 0,5 m de alt. ramos verdes, folhas cori\xe1ceas discolor verde escuras, ped\xfanculos lil\xe1s, flores roxas', u'occurrenceID': u'Arbusto c/ 0,5 m de alt. ramos verdes, folhas cori\xe1ceas discolor verde escuras, ped\xfanculos lil\xe1s, flores roxas', u'stateProvince': u'Brasil', u'rev': u'1-2ed980a27558b78c40dfc6302c30539b', u'comments': u'Faramea calyciflora', u'catalogNumber': u'RB', u'specificEpithet': u'Faramea', u'metadata': {u'creator': u'Raquel', u'created': 1430162160, u'modified': 1430162160, u'contact': u'raquel@cncflora.net', u'cont

In [35]:
print len(set(emails))
for email in set(emails):
    print email

24
lmoulton@cncflora.net ; Luiz Santos ; Raquel
lmoulton@cncflora.net ; luiz@cncflora.net ; Tainan Messina ; Raquel
lmoulton@cncflora.net ; luiz@cncflora.net ; Rodrigo Amaro ; Luiz Santos ; Raquel
luiz@cncflora.net ; Luiz Santos Filho ; Raquel
lmoulton@cncflora.net ; luiz@cncflora.net ; Luiz Santos ; Raquel ; Tainan Messina
tainan@cncflora.net ; Tainan Messina ; Raquel Negrão
tainan@cncflora.net ; Raquel ; Tainan Messina
raquel@cncflora.net ; luiz@cncflora.net ; Luiz Santos Filho ; Raquel Negrão
lmoulton@cncflora.net ; raquel@cncflora.net ; luiz@cncflora.net ; Raquel
lmoulton@cncflora.net ; raquel@cncflora.net ; luiz@cncflora.net ; Raquel ;  Elsie Franklin Guimarães
lmoulton@cncflora.net ; luiz@cncflora.net ; Luiz Santos ; Raquel ; Rodrigo Amaro
luiz@cncflora.net ; Luiz Santos Filho ; Raquel Negrão
lmoulton@cncflora.net ; raquel@cncflora.net ; luiz@cncflora.net ; Raquel ; Massimo Bovini
lmoulton@cncflora.net ; luiz@cncflora.net ; Raquel
raquel@cncflora.net ; luiz@cncflora.net ; Raquel
